In [ ]:
## Dummy Test Data Previous 
## Step 0.1: Import Libraries
import pulp as lp
import numpy as np 
from typing import List, Dict, Tuple

# --- Problem Size ---
COMPONENTS = [f'C_{i}' for i in range(1, 11)] 
ECUS = [f'E_{j}' for j in range(1, 5)]        
M = len(COMPONENTS) 

# --- O3 Weighting Parameters ---
LAMBDA_1 = 1.0   
LAMBDA_2 = 2.0   # Increased slightly to emphasize load balancing
LAMBDA_3 = 100.0 

# --- E2: ECU Capacity Data ---
# UPDATE: Slight buffer added to E_1 CPU (90 -> 95) just in case
MAX_CPU: Dict[str, float] = {'E_1': 95.0, 'E_2': 80.0, 'E_3': 50.0, 'E_4': 70.0}
MAX_RAM: Dict[str, float] = {'E_1': 200.0, 'E_2': 150.0, 'E_3': 50.0, 'E_4': 100.0}
ECU_BASE_COST: Dict[str, float] = {'E_1': 50.0, 'E_2': 40.0, 'E_3': 20.0, 'E_4': 35.0} # E_4 cost increased slightly due to upgrade

# --- C2: Component Requirement Data ---
REQ_CPU: Dict[str, float] = {
    'C_1': 30.0, 'C_2': 20.0, 'C_3': 5.0, 'C_4': 25.0, 'C_5': 10.0, 
    'C_6': 25.0, 'C_7': 15.0, 'C_8': 20.0, 'C_9': 10.0, 'C_10': 5.0
}
REQ_RAM: Dict[str, float] = {
    'C_1': 50.0, 'C_2': 10.0, 'C_3': 5.0, 'C_4': 40.0, 'C_5': 10.0, 
    'C_6': 15.0, 'C_7': 25.0, 'C_8': 30.0, 'C_9': 5.0, 'C_10': 10.0
}

# --- O1: Inter-Component Communication Volume ---
COMM_VOLUME: Dict[Tuple[str, str], float] = {
    ('C_1', 'C_4'): 10.0, ('C_4', 'C_1'): 10.0, 
    ('C_2', 'C_6'): 15.0, ('C_6', 'C_2'): 15.0, 
    ('C_2', 'C_9'): 4.0, ('C_9', 'C_2'): 4.0,
    ('C_4', 'C_8'): 5.0, ('C_8', 'C_4'): 5.0
}

# --- C3: Safety Isolation ---
CRITICAL_PAIRS_TO_SEPARATE: List[Tuple[str, str]] = [
    ('C_2', 'C_6') 
]

# --- C4: Hardware Requirements ---
REQ_HW: Dict[str, List[str]] = {
    'C_1': ['HW_GPU'],           # Camera
    'C_2': ['HW_HSM', 'HW_FLEX'],# Motor
    'C_3': [],                   
    'C_4': ['HW_GPU'],           # Sensor Fusion
    'C_5': [],                   
    'C_6': ['HW_HSM', 'HW_FLEX'],# Brake
    'C_7': [],                   
    'C_8': ['HW_GPU'],           # Parking
    'C_9': ['HW_HSM'],           # Power
    'C_10': []                   
}

ECU_HW_CAPABILITIES: Dict[str, List[str]] = {
    'E_1': ['HW_GPU', 'HW_HSM', 'HW_FLEX'], 
    'E_2': ['HW_FLEX', 'HW_HSM'],           
    'E_3': ['HW_HSM'],                      
    'E_4': ['HW_GPU']                       
}

# --- ASIL Definitions ---
ASIL_MAP = {'QM': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4}

# 1. Component ASIL Requirements
REQ_ASIL: Dict[str, str] = {
    'C_1': 'C', 'C_2': 'D', 'C_3': 'QM', 'C_4': 'B', 'C_5': 'QM',
    'C_6': 'D', 'C_7': 'QM', 'C_8': 'B', 'C_9': 'C', 'C_10': 'QM'
}

# 2. ECU ASIL Capabilities
ECU_ASIL_CAPABILITY: Dict[str, str] = {
    'E_1': 'D',  
    'E_2': 'D',  
    'E_3': 'B',  
    # UPDATE: E_4 upgraded from QM to B. 
    # This is the KEY fix. Now C_4 and C_8 (ASIL B + GPU) can run here.
    'E_4': 'B'   
}

print("Initial data structures defined (Adjusted for Feasibility).")


## Step 0.1: Import Libraries
import pulp as lp
import numpy as np 
from typing import List, Dict, Tuple

# --- Problem Size ---
COMPONENTS = [f'C_{i}' for i in range(1, 11)] 
ECUS = [f'E_{j}' for j in range(1, 5)]        
M = len(COMPONENTS) 

# --- O3 Weighting Parameters ---
LAMBDA_1 = 1.0   
LAMBDA_2 = 2.0   # Increased slightly to emphasize load balancing
LAMBDA_3 = 100.0 

# --- E2: ECU Capacity Data ---
# UPDATE: Slight buffer added to E_1 CPU (90 -> 95) just in case
MAX_CPU: Dict[str, float] = {'E_1': 95.0, 'E_2': 80.0, 'E_3': 50.0, 'E_4': 70.0}
MAX_RAM: Dict[str, float] = {'E_1': 200.0, 'E_2': 150.0, 'E_3': 50.0, 'E_4': 100.0}
ECU_BASE_COST: Dict[str, float] = {'E_1': 50.0, 'E_2': 40.0, 'E_3': 20.0, 'E_4': 35.0} # E_4 cost increased slightly due to upgrade

# --- C2: Component Requirement Data ---
REQ_CPU: Dict[str, float] = {
    'C_1': 30.0, 'C_2': 20.0, 'C_3': 5.0, 'C_4': 25.0, 'C_5': 10.0, 
    'C_6': 25.0, 'C_7': 15.0, 'C_8': 20.0, 'C_9': 10.0, 'C_10': 5.0
}
REQ_RAM: Dict[str, float] = {
    'C_1': 50.0, 'C_2': 10.0, 'C_3': 5.0, 'C_4': 40.0, 'C_5': 10.0, 
    'C_6': 15.0, 'C_7': 25.0, 'C_8': 30.0, 'C_9': 5.0, 'C_10': 10.0
}

# --- O1: Inter-Component Communication Volume ---
COMM_VOLUME: Dict[Tuple[str, str], float] = {
    ('C_1', 'C_4'): 10.0, ('C_4', 'C_1'): 10.0, 
    ('C_2', 'C_6'): 15.0, ('C_6', 'C_2'): 15.0, 
    ('C_2', 'C_9'): 4.0, ('C_9', 'C_2'): 4.0,
    ('C_4', 'C_8'): 5.0, ('C_8', 'C_4'): 5.0
}

# --- C3: Safety Isolation ---
CRITICAL_PAIRS_TO_SEPARATE: List[Tuple[str, str]] = [
    ('C_2', 'C_6') 
]

# --- C4: Hardware Requirements ---
REQ_HW: Dict[str, List[str]] = {
    'C_1': ['HW_GPU'],           # Camera
    'C_2': ['HW_HSM', 'HW_FLEX'],# Motor
    'C_3': [],                   
    'C_4': ['HW_GPU'],           # Sensor Fusion
    'C_5': [],                   
    'C_6': ['HW_HSM', 'HW_FLEX'],# Brake
    'C_7': [],                   
    'C_8': ['HW_GPU'],           # Parking
    'C_9': ['HW_HSM'],           # Power
    'C_10': []                   
}

ECU_HW_CAPABILITIES: Dict[str, List[str]] = {
    'E_1': ['HW_GPU', 'HW_HSM', 'HW_FLEX'], 
    'E_2': ['HW_FLEX', 'HW_HSM'],           
    'E_3': ['HW_HSM'],                      
    'E_4': ['HW_GPU']                       
}

# --- ASIL Definitions ---
ASIL_MAP = {'QM': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4}

# 1. Component ASIL Requirements
REQ_ASIL: Dict[str, str] = {
    'C_1': 'C', 'C_2': 'D', 'C_3': 'QM', 'C_4': 'B', 'C_5': 'QM',
    'C_6': 'D', 'C_7': 'QM', 'C_8': 'B', 'C_9': 'C', 'C_10': 'QM'
}

# 2. ECU ASIL Capabilities
ECU_ASIL_CAPABILITY: Dict[str, str] = {
    'E_1': 'D',  
    'E_2': 'D',  
    'E_3': 'B',  
    # UPDATE: E_4 upgraded from QM to B. 
    # This is the KEY fix. Now C_4 and C_8 (ASIL B + GPU) can run here.
    'E_4': 'B'   
}

print("Initial data structures defined (Adjusted for Feasibility).")